# Admissions markets with a single score and MNL choice
Let’s consider a special kind of admissions market that has not received much attention in the school-choice literature but approximates the admissions procedure used in many systems around the world.

In the *single-score* model, all schools share the same ranking over the students. Here are a few situations in which a single-score system may arise:
- A centralized college admissions market similar to that used in China, where college admissions are determined largely by a single test score.
- Students are scored using a consistent linear combination of various dimensions of student characteristics, e.g. 70% test scores, 20% GPA, 10% letter of recommendation.
- Single tiebreaking, in which a central agency assigns (by lottery) an admissions priority ranking to each student and students are matched to their favorite schools in priority order.

Regardless of the device used to generate the single scores, taking percentile scores over the distribution allows us to assume that the scores are uniformly distributed on the interval $[0,1]$. 

We also need to account for students' choice of school. In this model, students use the *multinomial logit choice* model. This means that each school has a given *quality* $\delta_c$. Each student is admitted to a certain set of schools $C^\#$, and from this set, she chooses to attend school $c$ with probability

$$\frac{\exp \delta_c}{\sum_{d \in C^\#} \exp \delta_d}$$

For convenience, let $\gamma_c \equiv \exp \delta_c > 0$. Since the equation is homogeneous in $\gamma$, we may assume without loss of generality that $\sum_c \gamma_c = 1$. I will refrain from this assumption in the explanatory text but make full use of it in the code.

## Demand function
The first quantity we need to define is the *demand function,* which indicates how many students will attend each school given the school's quality $\gamma$ and some information about the school's selectivity. Specifically, each school sets a *cutoff* $p_c$ and admits all students whose score exceeds the cutoff. The demand function $D_c(\gamma, p)$ indicates how many students come to school $c$, representated as a fraction of the total number of students in the market. Observe that $D_c$ is a function not just of $p_c$ but also of the cutoffs at all the other schools, since if another school becomes more selective, the demand at $c$ may increase.

Let's figure out this function. First, sort the schools by cutoff, i.e. so that

$$p_1 \leq p_2 \leq \dots \leq p_{|C|}$$

Then there are only $|C| + 1$ possible consideration sets for each student: 

| Symbol          | Consideration&nbsp;set                          | Probability           |
|-----------------|--------------------------------------------|-----------------------|
| $C_{[0]}$       | $\varnothing$                              | $p_1$                 |
| $C_{[1]}$       | $\left\{ c_1 \right\}$                     | $p_2 - p_1$           |
| $C_{[2]}$       | $\left\{ c_1, c_2 \right\}$                | $p_3 - p_2$           |
| $\vdots$        | $\vdots$                                   | $\vdots$              |
| $C_{[|C| - 1]}$ | $\left\{ c_1, \dots, c_{|C| - 1} \right\}$ | $p_{|C|} - p_{|C|-1}$ |
| $C_{[|C|]}$     | $\left\{ c_1, \dots, c_{|C|} \right\}$     | $1 - p_{|C|}$         |

This greatly simplifies the demand function. Letting $p_{|C|+1} \equiv 1$,

$$\begin{equation}D_c = \sum_{d=c}^{|C|} 
\underbrace{\frac{{\gamma_c}}{ \sum_{i=1}^d {\gamma_i}}}_{\substack{\text{prob. of choosing  }\\ c\text{ from assortment}}} 
\overbrace{\left(p_{d+1} - p_{d}\right)}^{\substack{\text{prob. of having}\\ \text{assortment }C_{[d]}}} \end{equation}$$

If at least one school has $p_c = 0$, then every student can get in somewhere, and $\sum_c D_c = 1$. Generally, there are $p_1$ students who get in nowhere, and $\sum_c D_c = 1 - p_1$. 

The demand function is *piecewise linear* in $p$ (piecewise because the ordering of the $p_c$ values changes). Expanding the equation above,

$$D_c = \gamma_c \left[ \left(\frac{-1}{\sum_{i=1}^c \gamma_i}\right) p_c
+ \left(\frac{1}{\sum_{i=1}^{c} \gamma_i} - \frac{1}{\sum_{i=1}^{c+1} \gamma_i} \right) p_{c+1}
+ \left(\frac{1}{\sum_{i=1}^{c+1} \gamma_i} - \frac{1}{\sum_{i=1}^{c+2} \gamma_i}\right) p_{c+2}
+ \cdots
+ \left(\frac{1}{\sum_{i=1}^{|C|-1} \gamma_i} - \frac{1}{\sum_{i=1}^{|C|} \gamma_i}\right) p_{|C|}
+ \frac{1}{\sum_{i=1}^{|C|} \gamma_i}
\right]$$

and we can see that the demand vector is defined by the system

$$ D = A p + \frac{1}{\sum_{i=1}^{|C|} \gamma_i}\gamma$$

where 

$$A \equiv \begin{bmatrix}
\gamma_1 \left( \frac{-1}{\gamma_1} \right) & \gamma_1 \left(\frac{1}{\gamma_1} - \frac{1}{\gamma_1 + \gamma_2} \right) & \gamma_1 \left(\frac{1}{\gamma_1 + \gamma_2} - \frac{1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_1 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - \frac{1}{\sum_{i=1}^{|C|}\gamma_i}  \right)  \\
0 & \gamma_2 \left( \frac{-1}{\gamma_1 + \gamma_2} \right) & \gamma_2 \left(\frac{1}{\gamma_1 + \gamma_2} - \frac{1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_2 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - \frac{1}{\sum_{i=1}^{|C|}\gamma_i} \right)  \\
0 & 0 & \gamma_3 \left( \frac{-1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_3 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - 1 \right)  \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 &  \gamma_{|C|} \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} -\frac{1}{\sum_{i=1}^{|C|}\gamma_i}  \right)  \\
\end{bmatrix}$$

Since $\gamma > 0$, $A\in \mathbb{R}^{|C| \times |C|}$ is strictly triangular, hence invertible. 

## Appeal

It is also interesting to consider the *appeal* of a school's entering class, or the integral of scores over the set of admitted students (Azevedo and Leshno 2016). This is not necessarily the school's objective function, because schools may value an abstract notion of selectivity or students' tuition dollars higher than this value. 

The average score of a student with consideration set $C_{[d]}$ is $\frac{1}{2}\left(p_{d+1} + p_d\right)$, so the appeal at $c$ is

$$L_c = \frac{1}{2}\sum_{d=c}^{|C|} 
\frac{{\gamma_c}}{ \sum_{i=1}^d {\gamma_i}} 
\left(p_{d+1} - p_{d}\right)\left(p_{d+1} + p_{d}\right) =
\frac{1}{2}\sum_{d=c}^{|C|} 
\frac{{\gamma_c}}{ \sum_{i=1}^d {\gamma_i}} 
\left(p_{d+1}^2 -  p_{d}^2\right)$$

By comparison with the expression for $D$, the quality vector is given by 

$$ L = \frac{1}{2} A p.^2 + \frac{1}{2} \gamma$$

where I have used the strange notation $p.^2 = (p_1^2, \dots, p_{|C|}^2)$ for the entrywise square of $p$.

## Computation

Let's get a computational environment going. First I define a container for the market information (capacity is discussed later), then the demand and appeal functions. I also define a function that returns the matrix $A$ discussed above; note that $A$ depends on $p$ via permutation. Finally, I define a market that computes the equilibrium for a given market using a generic t&acirc;tonnement procedure.

In [255]:
using LinearAlgebra

"""
Contains static information about a school-choice market.
"""
struct Market{T<:AbstractFloat}
    qualities ::Array{T, 1}    # δ
    capacities::Array{T, 1}    # q
    gamma     ::Array{T, 1}    # γ
end

"""
    Market(qualities, capacities)

Instantiate a school-choice market. `gamma` is normalized to sum to one.
"""
function Market(qualities::Array{T, 1}, capacities::Array{T, 1}) where {T<:AbstractFloat}
    length(qualities) == length(capacities) || return throw(DimensionMismatch)
    gamma = exp.(qualities)
    gamma /= sum(gamma)
    return Market{T}(qualities, capacities, gamma)
end
    
    
"""
    Market(m)

Generate a random market with m schools.
"""
function Market(m::Int) 
    qualities = rand(m)
    capacities = 2 * rand(m)/m
    return Market(qualities, capacities)
end


"""
    demand(market, cutoffs)

Return demand for each school given a set of cutoffs and ignoring capacity, using
multinomial logit choice model with one student profile and a single test score.
"""
function demand(market      ::Market,
                cutoffs     ::AbstractArray{<:AbstractFloat, 1};
                )::AbstractArray{<:AbstractFloat, 1}
    
    m = length(market.qualities)
    demands = zeros(m)

    sort_order = sortperm(cutoffs)
    cutoffs[sort_order]

    γ = exp.(market.qualities)
    demands = zeros(m)

    consideration_set_probabilities = diff([cutoffs[sort_order]; 1])

    for c in 1:m, d in c:m     # For each score threshold
        demands[sort_order[c]] += consideration_set_probabilities[d] *
                                  γ[sort_order[c]] / sum(γ[sort_order[1:d]])
    end

    return demands
end


"""
    appeal(market, cutoffs)

Return appeal of entring class at each school given a set of cutoffs, using
multinomial logit choice model with one student profile and a single test score.
"""
function appeal(market      ::Market,
                cutoffs     ::AbstractArray{<:AbstractFloat, 1};
                )::AbstractArray{<:AbstractFloat, 1}
    
    m = length(market.qualities)
    demands = zeros(m)

    sort_order = sortperm(cutoffs)
    cutoffs[sort_order]

    γ = exp.(market.qualities)
    appeals = zeros(m)

    diff_of_squares = diff([cutoffs[sort_order]; 1] .^ 2)

    for c in 1:m, d in c:m     # For each score threshold
        appeals[sort_order[c]] += diff_of_squares[d] *
                                  γ[sort_order[c]] / sum(γ[sort_order[1:d]])
    end

    return appeals / 2
end


"""
    demandmatrix(market, cutoffs)

Returns the matrices `A` and the permutation `sort_order` such that

````
demand(market, p)[sort_order] == A * p[sort_order] + market.gamma[sort_order]
````

and

````
appeal(market, p)[sort_order] == (A * p[sort_order] .^2 + market.gamma[sort_order])/2
````
"""
function demandmatrix(market, cutoffs)
    m = length(market.qualities)
    
    sort_order = sortperm(cutoffs)
    
    A = UpperTriangular(zeros(m, m))
    
    for c in 1:m
        A[c, c] = -market.gamma[sort_order[c]] / sum(market.gamma[sort_order[1:c]])
    end
    
    for i in 1:m, j in i+1:m
        A[i, j] = market.gamma[sort_order[i]] *
                    ( 1/sum(market.gamma[sort_order[1:j-1]]) -
                      1/sum(market.gamma[sort_order[1:j]]) )
    end
    
    return A, sort_order
end


"""
    quickeq(market, demand)

Very low-tech tatonnement procedure for finding equibilibrium.
"""
function quickeq(market::Market, demand::Function; maxit::Int=100)
    p = rand(length(market.qualities))
    
    for i in 1:maxit
        p = max.(0, p + .5 * (demand(market, p) - market.capacities) / i^0.001)
#         @show p
    end
    
    return p
end

quickeq

## Checking the appeal and demand functions

Check that this function works as described:

In [268]:
mkt = Market(5)
p = rand(5)

A, sort_order = demandmatrix(mkt, p)

demand(mkt, p)[sort_order] ≈ A * p[sort_order] + mkt.gamma[sort_order]

true

Alternatively, 

In [269]:
inv_sort_order = invperm(sort_order)
B = A[inv_sort_order, inv_sort_order]

demand(mkt, p) ≈ B * p + mkt.gamma

true

The following fact is handy, because inverting a triangular matrix is easier than inverting a square matrix:

In [258]:
inv(A)[inv_sort_order, inv_sort_order] ≈ inv(B)

true

Same check for the appeal function:

In [270]:
appeal(mkt, p)[sort_order] ≈ (A * p[sort_order] .^2 + mkt.gamma[sort_order])/2 &&
appeal(mkt, p) ≈ (A[inv_sort_order, inv_sort_order] * p .^2 + mkt.gamma)/2

true

In [271]:
taipei = Market([2., 1., 2.], [0.2, 0.3, 0.1])
p_star = quickeq(taipei, demand)

3-element Vector{Float64}:
 0.6264241117626806
 0.3999999999955685
 0.7632120558798249

In [273]:
A, sort_order = demandmatrix(taipei, p_star)
inv_sort_order = invperm(sort_order)
B = A[inv_sort_order, inv_sort_order]

3×3 Matrix{Float64}:
 -0.731059   0.0   0.30874
  0.731059  -1.0   0.113579
  0.0        0.0  -0.422319

In [277]:
B * diagm(p_star)

3×3 Matrix{Float64}:
 -0.457953   0.0   0.235634
  0.457953  -0.4   0.0866849
  0.0        0.0  -0.322319

In [279]:
inv(B)

3×3 Matrix{Float64}:
 -1.36788  -0.0  -1.0
 -1.0      -1.0  -1.0
  0.0       0.0  -2.36788

In [348]:
mkt = Market(6)
p_star = quickeq(mkt, demand)
A, sort_order = demandmatrix(mkt, p_star)
inv_sort_order = invperm(sort_order)
B = A[inv_sort_order, inv_sort_order]

6×6 Matrix{Float64}:
 -1.0   0.0268619   0.131002   0.65426   0.0612728   0.0286863
  0.0  -0.215275    0.0        0.0       0.0         0.0
  0.0   0.0473973  -0.378902   0.0       0.108115    0.0506164
  0.0   0.0508319   0.2479    -0.65426   0.115949    0.0542843
  0.0   0.0499439   0.0        0.0      -0.285337    0.053336
  0.0   0.0402398   0.0        0.0       0.0        -0.186923

In [349]:
sum(mkt.capacities)

1.1005894138924703

In [350]:
inv(B)

6×6 Matrix{Float64}:
 -1.0  -1.0      -1.0     -1.0      -1.0      -1.0
  0.0  -4.64523  -0.0     -0.0      -0.0      -0.0
  0.0  -1.0      -2.6392  -0.0      -1.0      -1.0
  0.0  -1.0      -1.0     -1.52845  -1.0      -1.0
  0.0  -1.0       0.0      0.0      -3.50463  -1.0
  0.0  -1.0       0.0      0.0       0.0      -5.3498

In [351]:
p_star

6-element Vector{Float64}:
 0.0
 0.7757585077985446
 0.04131307427433985
 0.023620650919357146
 0.14126544033978675
 0.3664662213750571

In [352]:
mkt.capacities - demand(mkt, p_star)

6-element Vector{Float64}:
  0.10059137008893737
 -1.0433964235806448e-6
 -3.562377077215295e-7
 -3.4855833846991757e-7
 -3.4569628923342144e-7
  1.3769229199878108e-7

In [353]:
B\(mkt.capacities - mkt.gamma)

6-element Vector{Float64}:
 -0.10058941389247018
  0.775763354610248
  0.04131526585881682
  0.023622791309795686
  0.14126755758150203
  0.3664665281453254

## Comparative statics

An area of interest in economics is *comparative statics,* or numbers that signify the tendencies or incentives available to the competitive actors in a market. 

Given $D$, $p$, and $\gamma$, we can use the relations $D = Ap + \gamma$ and $L = \frac{1}{2} A p.^2 + \frac{1}{2}\gamma$ to make a number of interesting observations about the market.


### Price effects
The change in demand in response to a change in cutoffs is the Jacobian of the demand function:
$$\mathbf{J}_p D = A = \begin{bmatrix}
\gamma_1 \left( \frac{-1}{\gamma_1} \right) & \gamma_1 \left(\frac{1}{\gamma_1} - \frac{1}{\gamma_1 + \gamma_2} \right) & \gamma_1 \left(\frac{1}{\gamma_1 + \gamma_2} - \frac{1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_1 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - \frac{1}{\sum_{i=1}^{|C|}\gamma_i}  \right)  \\
0 & \gamma_2 \left( \frac{-1}{\gamma_1 + \gamma_2} \right) & \gamma_2 \left(\frac{1}{\gamma_1 + \gamma_2} - \frac{1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_2 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - \frac{1}{\sum_{i=1}^{|C|}\gamma_i} \right)  \\
0 & 0 & \gamma_3 \left( \frac{-1}{\gamma_1 + \gamma_2 + \gamma_3} \right) & \cdots &  \gamma_3 \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} - 1 \right)  \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 &  \gamma_{|C|} \left(\frac{1}{\sum_{i=1}^{|C| - 1}\gamma_i} -\frac{1}{\sum_{i=1}^{|C|}\gamma_i}  \right)  \\
\end{bmatrix}$$
The diagonal is negative, meaning that each school's demand is decreasing in its cutoff (downward-sloping demand curves). The entries above the diagonal are positive, while those below the diagonal are zero. This means that each school $c$'s demand is increasing in the cutoffs of the *more-selective* schools, but the cutoffs of *less-selective schools* have no local effect on the demand at $c$.

Intuitively, this means that if all schools are equally preferable, a highly selective school has more market power than the others: If it increases its cutoff, it will cause many students to move onto another school. On the other hand, a school $c'$ that is less preferable than $c$ cannot affect $D_c$'s demand by changing its own cutoff, because any student currently admitted to $c$ was already admitted to $c'$, and chose $c$ instead. 

Now consider the change in *appeal* in response to a change in cutoffs:


## Dynamic admissions

The market in consideration has four parameters:
 1. The vector of school capacities $q$
 2. The vector of school qualities $\gamma$ (or $\delta$)
 3. The demand vector $D$
 4. The capacity of each school $q$
 
The first three parameters are related by the demand function $D(\gamma, p)$.

Depending on the design of the market itself, the demand function may or may not satisfy the equilibrium condition
$$\begin{align} D_c &\leq q_c, \quad \forall c \\
D_c &= q_c, \quad \forall c: p_c > 0\end{align}$$

The first condition says that no school's demand exceeds its capacity. The second says that if a school is rejecting students, it must be at full capacity; equivalently, if a school has remaining capacity, it will try to minimize the amount of empty seats by lowering its cutoff all the way to zero.

It can be shown that if the parameters satisfy the relation above, then the resultant allocation of students is *stable,* meaning for every student who is *not* matched to her favorite school, each of the schools she prefers to her match has already filled its capacity with better students.  
 
Here are a few scenarios in which we can expect an admissions market to satisfy the equilibrium condition above.
- A *centralized school-choice process* such as the deferred acceptance algorithm, in which students submit their preference lists to a central agency that computes a stable match.
- A *semicentralized* admissions procedure like that used in Korea, in which the government determines a quota (i.e. capacity) of students who may be admitted to each school, and schools steadily admit students with lower and lower scores over the course of several admissions rounds until they fill their quota. (In the idealized case, this is actually a decentralized deferred acceptance algorithm; in practice, there is variation in scoring practices and limits on how many schools each student can apply to that prevent the equilibrium from being achieved exactly.)
- A *decentralized* admissions procedure like that used by private universities in the US. In this case, schools designate their own capacity, which is not necessarily a physical limitation on the number of students it can receive but rather a way of maintaining the school's selective reputation. In this case, schools must observe their demand from year over year and adjust their admissions cutoffs accordingly. If the adjustment process fits certain regularity conditions, then this process converges to the equilibrium defined above. (In fact, a version of this process is how the function `quickeq()` computes equilibria.)

These observations are useful because if we can assume the market is at equilibrium, we can reduce the number of parameters:
- If $q$ and $\gamma$ are given, then there is only one equilibrium cutoff vector $p$, which we can find using `quickeq()`.
- If $\gamma$ and $p > 0$ are given, and we are told that the market is at equilibrium, then we can assume that $q = D(\gamma, p)$, which we can find using the `demand()` function.
- If $D$ and $p$ are given, and we are told that the market is at equilibrium, then we can solve for $\gamma$, as described [in a separate notebook](https://github.com/maxkapur/StudentPrefsRevOpt/blob/main/2_ReverseOptimization.ipynb). If $p>0$, we can additionally assume that $q = D$. 

In [147]:
B

5×5 Matrix{Float64}:
 -0.234931   0.0   0.0239967   0.0769739   0.0
  0.109574  -1.0   0.0364486   0.116916    0.53359
  0.0        0.0  -0.15192     0.0         0.0
  0.0        0.0   0.0497757  -0.327645    0.0
  0.125357   0.0   0.0416985   0.133756   -0.53359

In [34]:
UpperTriangular(zeros(12 ,12))[:, [1, 5, 6]]

12×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

## References

- Abdulkadiroğlu, Atila, Yeon-Koo Che, and Yosuke Yasuda. 2015. &ldquo;Expanding &lsquo;Choice&rsquo; in School Choice.&rdquo; *American Economic Journal: Microeconomics* 7, no. 1 (Feb.): 1&ndash;42.